In [1]:
import numpy as np 
import quadprog
import matplotlib.pyplot as plt 
import pandas as pd
import requests
import ast
import time
import schedule
from datetime import date
from datetime import datetime

In [ ]:
import requests

url = "https://api.tomorrow.io/v4/timelines?location=42.3478%2C%20-71.0466&fields=temperature&units=metric&timesteps=1h&startTime=now&endTime=nowPlus6h&apikey=VYLHerX2Zbk6nBvVo4VTkpGg2JFJTfw3"

headers = {
    "Accept": "application/json",
    "Accept-Encoding": "gzip"
}

response = requests.get(url, headers=headers)


In [ ]:
('location', '30.2955535, -97.7398146'),#Hemphillpark2 
('location', '30.3359239, -97.7126299'),#highland 
('location', '30.3081971, -97.7278405'),#shipe neighbor 
('location', '30.2858333, -97.7338333'),#bridge3 
('location', '30.2834167, -97.7344444'),#bridge4
('location', '30.2871667, -97.7341111'),#bridge5
('location', '30.2879080, -97.7341380'),#Campus2
('location', '30.2885374, -97.7346483'),#bridge6
('location', '30.2886667, -97.7337500'),#bridge2
('location', '30.2905109, -97.7305199'),#Eastwoods neighborhood 
('location', '30.2949913, -97.7394333'),#Hempillpark3
('location', '30.3129327, -97.7337902'),#Triangle pond
('location', '30.3004298, -97.7245227'),#Hancock receation center
('location', '30.3213002, -97.7230546'),#Chesterfield
('location', '30.3265250, -97.7199113'),#Relly pond
('location', '30.3337866, -97.7140881'),#Kenniston
('location', '30.3412541, -97.7150030'),#Crestland Triangle
('location', '30.2924810, -97.7363710'),#Campus1
('location', '30.2914444, -97.7353333')]#bridge1

In [ ]:
location=[('location', '30.28412, -97.73118'), #PRC
('location', '30.2978508, -97.7390084')] #Hemphillpark 


fields=[('fields', 'precipitationProbability'),
('fields', 'rainIntensity')]

timesteps=[('timesteps', '1d'),('timesteps', '1h')]

In [ ]:
def collect_tomorrow_API_forecast_data(i):
    set_location=location[i]
    for j in range(0, len(fields)):
        set_field=fields[j]
        for k in range(0, len(timesteps)):
            set_timestep=timesteps[k]
            pre_raindata=pd.read_csv("/Users/future_water_system/Desktop/crawling/raindata.csv")
            # set file name - define i,j,k 
            if k == 0:
                name_timestep = '1d'
            elif k == 1:
                name_timestep ='1h'

            if j == 0:
                name_field= 'probability'
            elif j ==1:
                name_field='intensity'

            date_value=datetime.now().strftime('%Y%m%d')
            time_value=datetime.now().strftime('%H%M%S')

            params = (set_location,set_field,set_timestep,('units', 'metric'),('apikey', 'VYLHerX2Zbk6nBvVo4VTkpGg2JFJTfw3'),)
            headers = {
            "Accept": "application/json",
            "Accept-Encoding": "gzip"}

            response = requests.get('https://api.tomorrow.io/v4/timelines', params=params, headers=headers)
            time.sleep(10)

            #change to dictionary format
            content=response.content
            dict_str = content.decode("UTF-8")
            my_data = ast.literal_eval(dict_str)
            sub=my_data

            rain_data = pd.DataFrame([[date_value,time_value,set_location,name_field,name_timestep,sub]],columns=['date','time','loc','field','timestep','value'])
            now_raindata=pd.concat([pre_raindata,rain_data])
            now_raindata.to_csv("/Users/future_water_system/Desktop/crawling/raindata.csv", index=False)


                        
            
                
                

In [ ]:
#Tomorrow.io API https://docs.tomorrow.io/reference/get-timelines
# use schedule module to do work



def adaptive_crawling(i):
    loc=location[i]
    params = (
            loc,
            ('fields', 'precipitationProbability'),
            ('timesteps', '1d'),
            ('units', 'metric'),
            ('apikey', 'VYLHerX2Zbk6nBvVo4VTkpGg2JFJTfw3'),)
    headers = {
    "Accept": "application/json",
    "Accept-Encoding": "gzip"}

    response = requests.get('https://api.tomorrow.io/v4/timelines', params=params, headers=headers)

    #change to dictionary format
    content=response.content
    dict_str = content.decode("UTF-8")
    my_data = ast.literal_eval(dict_str)
    sub=my_data
    sub=sub['data']['timelines']

    sub_str=sub[0]
    interval=sub_str['intervals']

    #Select value in dictionbary 
    rainStartTime=[]
    value=np.zeros((len(interval),))

    for i in range(0,len(interval)):
        interval_str=interval[i]
        rainStartTime.append(interval_str['startTime'])
        value[i]=interval_str['values']['precipitationProbability']
        
        return rainStartTime[0],value[0]
        


def crawling_job():
    
    while(True):

    for i in range(0,len(location)):

        today_date,today_rainprob=adaptive_crawling(i)

        if today_rainprob >= 10:
            
            collect_tomorrow_API_forecast_data(i)

    time.sleep(3600)

     

In [ ]:

#Adaptive crawling -pretest for collecting date 
schedule.every().day.at("00:00").do(crawling_job) 
     
while True:
    schedule.run_pending()
    time.sleep(1)
       